In [1]:
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM,TimeDistributed, Embedding
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop, Adam, SGD
import sys
import numpy as np

Using TensorFlow backend.


In [2]:
fname = 'dostoevsky.txt'
text = open(fname, 'r', encoding='utf-8').read()
print('corpus length:', len(text))

corpus length: 9469628


In [4]:
chars = sorted(list(set(text)))
print('unique chars:', len(chars))
char_idx = dict((c, i) for i, c in enumerate(chars))
idx_char = dict((i, c) for i, c in enumerate(chars))

bigrams = []
for i in range(0, len(text)-1, 1):
    bigrams.append(text[i] + text[i+1])
bigrams = sorted(list(set(bigrams)))
print('unique bigrams:', len(bigrams))
bigram_idx = dict((c, i) for i, c in enumerate(bigrams))
idx_bigram = dict((i, c) for i, c in enumerate(bigrams))

trigrams = []
for i in range(0, len(text)-2, 1):
    trigrams.append(text[i] + text[i+1] + text[i+2])
trigrams = sorted(list(set(trigrams)))
print('unique trigrams:', len(trigrams))
trigram_idx = dict((c, i) for i, c in enumerate(trigrams))
idx_trigram = dict((i, c) for i, c in enumerate(trigrams))

words = text.split()
print('word count:', len(words))
words = sorted(list(set(words)))
print('unique words:', len(words))

unique chars: 162
unique bigrams: 3566
unique trigrams: 26082
word count: 1528727
unique words: 165930


In [5]:
batch_size = 1024 # decrease if you have "Failed to allocate memory" error when start training
track_size = len(text) // batch_size
tracks = ['' for i in range(batch_size)]
for i in range(0, track_size):
    for track in range(batch_size):
        tracks[track] += text[track * track_size + i]

# Let's see what we've got
print(tracks[4][:1000])

 только, что сын его, воспитывавшийся сначала у графа, а потом в лицее, окончил тогда курс наук девятнадцати лет от роду. Я написал об этом к Ихменевым, а также и о том, что князь очень любит своего сына, балует его, рассчитывает уже и теперь его будущность. Всё это я узнал от товарищей-студентов, знакомых молодому князю. В это-то время Николай Сергеич в одно прекрасное утро получил от князя письмо, чрезвычайно его удивившее...
   Князь, который до сих пор, как уже упомянул я, ограничивался в сношениях с Николаем Сергеичем одной сухой, деловой перепиской, писал к нему теперь самым подробным, откровенным и дружеским образом о своих семейных обстоятельствах: он жаловался на своего сына, писал, что сын огорчает его дурным своим поведением; что, конечно, на шалости такого мальчика нельзя еще смотреть слишком серьезно (он видимо старался оправдать его), но что он решился наказать сына, попугать его, а именно: сослать ого на некоторое время в деревню, под присмотр Ихменева. Князь писал, что 

In [7]:
gram_idx = [char_idx, bigram_idx, trigram_idx]
idx_gram = [idx_char, idx_bigram, idx_trigram]

#converts to indexes of chars, bigrams, trigrams
def grams(tracks, n=1):
    indexed = []
    for t in tracks:  
        track = []
        for i in range(0, len(t)-n+1, n):
            gram = ''
            for j in range(n):
                gram += t[i+j]
            idx = gram_idx[n-1][gram]
            track.append(idx)
        indexed.append(track)
    return indexed

indexed = grams(tracks, 1)
vocab_size = len(gram_idx[0])
print('Vocabulary done: ', vocab_size)

Vocabulary done:  162


In [8]:
seq_len = 40 #length of sequence in a batch

def onehot(n):
    v = [0 for i in range(vocab_size)]
    v[n] = 1
    return v

#build order for batches for stateful lstm
def vectorize(tracks):
    track_size = len(tracks[0])
    X = []
    y = []
    for i in range(0, track_size - seq_len + 1, seq_len):
        for t in tracks:
            X.append(t[i:i + seq_len])
            target = [onehot(c) for c in t[i+1:i + seq_len + 1]]
            y.append(target)
    return X, y
        
x,y = vectorize(indexed)
print('Number of training samples', len(x))
print('Number of training labels', len(y))
print('Label sequence length', len(y[0]))
print('Label character one-hot vector length', len(y[0][0]))

Number of training samples 236544
Number of training labels 236544
Label sequence length 40
Label character one-hot vector length 162


In [9]:
cells = 512
drop = 0.2
embed = 30 # size of character embedding
layers = 2
lr = 0.01
clip = 5.0 # gradient clipping to prevent exploding gradients
stateful=True # maintain layer state between batches

model = Sequential()
model.add(Embedding(vocab_size, embed, batch_input_shape=(batch_size, seq_len)))
for l in range(layers):
    model.add(LSTM(cells, return_sequences=True, stateful=stateful, dropout=drop))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
optimizer = Adam(lr, clipnorm=clip)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1024, 40, 30)            4860      
_________________________________________________________________
lstm_1 (LSTM)                (1024, 40, 512)           1112064   
_________________________________________________________________
lstm_2 (LSTM)                (1024, 40, 512)           2099200   
_________________________________________________________________
dense_1 (Dense)              (1024, 40, 162)           83106     
_________________________________________________________________
activation_1 (Activation)    (1024, 40, 162)           0         
Total params: 3,299,230
Trainable params: 3,299,230
Non-trainable params: 0
_________________________________________________________________


In [10]:
def get_callbacks(filepath, patience=5):
    learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=patience, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    es = EarlyStopping('loss', verbose=1, min_delta=0.02, patience=patience, mode="min")
    return [learning_rate_reduction, es]

In [11]:
def sample(preds, temperature=0.5):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
def predictive_model(main_model): # change batch size to 1 for work with one sequence
    model = Sequential()
    model.add(Embedding(vocab_size, embed, batch_input_shape=(1, seq_len)))
    for l in range(layers):
        model.add(LSTM(cells, return_sequences=True, stateful=stateful, dropout=drop))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    optimizer = Adam(lr, clipnorm=clip)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer)
    old_weights = main_model.get_weights()
    model.set_weights(old_weights)
    return model

In [21]:
# generate new chars from model
def test(model, l=500, seed = None, t=0.5):
    start_from = np.random.randint(len(text)-seq_len)+seq_len
    seed_string = text[start_from:start_from + seq_len*3] if seed is None else seed
    print('\n\nSeed:  ', seed_string)
    print('----')
    sys.stdout.write(seed_string)
    prmodel = predictive_model(model)
    for i in range(l):
        prmodel.reset_states()
        padlen = (len(seed_string) // seq_len +1) * seq_len
        seed_string = seed_string.rjust(padlen)[-seq_len*3:]
        test_tracks = [seed_string]
        tidx = grams(test_tracks)
        xt, _ = vectorize(tidx)
        preds = prmodel.predict(np.array(xt), batch_size=1, verbose=0)
        preds = preds[-1][-1] # last symbol of last sequence
        next_item = idx_char[sample(preds, t)]
        seed_string = seed_string + next_item
        sys.stdout.write(next_item)
        sys.stdout.flush()    

In [15]:
for iteration in range(1, 51):
    print('\nIteration', iteration)
    model_name = 'char_%s_%d_%d_%.1f_%d.h5' % (fname, layers, cells, drop, iteration)
    history=model.fit(
        np.array(x), np.array(y), 
        batch_size=batch_size, 
        epochs=1, 
        verbose=1, 
        shuffle=False,
        callbacks=get_callbacks(filepath=model_name)
    )
    model.save_weights(model_name, overwrite=True)
    model.reset_states()
    if iteration%3 == 0:
        test(model)


Iteration 1
Epoch 1/1
236544/236544 [==============================] - 196s - loss: 3.3098   

Iteration 2
Epoch 1/1
236544/236544 [==============================] - 188s - loss: 3.1676   

Iteration 3
Epoch 1/1
236544/236544 [==============================] - 191s - loss: 2.5591   
>>>  икальным объяснением, несмотря на то что дело плевое; я знаю его еще с Петербурга. К тому же весь анекдот делает только 
----
икальным объяснением, несмотря на то что дело плевое; я знаю его еще с Петербурга. К тому же весь анекдот делает только на не воглану и вот двадь не сопривила не на прокоть пробовореться на солать не могда уго всего же самовеле дервать,  потодал
 что старет придорил отанила спа не прогость же того и не сот отводил все ста стастве. Прегавове,  скоронить е всё логда на с нимененно послей на проемала не не стольто вы мне мни сам, кня на старате телова пословаеть так на на смество миже все преседь слуго все в неста не сами призчите вот, не на тогда не закорорно как трязь потисту со

In [25]:
seed = "У нас в Малом зале до сих пор проходят"
test(model, 300, seed, 0.1)
test(model, 300, seed, 0.5)
test(model, 300, seed, 0.7)



Seed:   У нас в Малом зале до сих пор проходят
----
У нас в Малом зале до сих пор проходят с ним с тобой и подозревали и подозревали и при всех столь и под конец просто подозрения и просто подозревали и при своей стороны и принять своего простодушного старика и подозревали и просто подозревали и подозревали и под конец в своем положении и принять в себе старика, но в том, что он всегда п

Seed:   У нас в Малом зале до сих пор проходят
----
У нас в Малом зале до сих пор проходят на свою разом и не после прежнего правда с тревожностью, выразился и смотрел на положении. Но просто отвечала она принять меня и сказал его и пред нею, как бы во всех своего проклятий принес в картину и стал простодушное ветром и воздух не понимаю, что не давать предстоящего и судорога. Она убил с 

Seed:   У нас в Малом зале до сих пор проходят
----
У нас в Малом зале до сих пор проходят картины. Теперь сейчас они остановившись пошле дорогу совсем и почти неожиданное  своего  затрастно   на  собой  в  том,  ч